In [ ]:
import intake_tools
import intake
from pystac import Catalog, Item, Asset, MediaType, CatalogType
import stac_tools
import logging

logger = logging.getLogger("PlayWithStack")

logger.setLevel(logging.ERROR)

In [2]:
cat = intake.open_catalog("../catalog.yaml")

In [3]:
stac_cat = Catalog("nextgems", "nextGEMS simulations")


def create_subcat(cat, child, position):
    parent = stac_tools.get_subcat(stac_cat, position)
    child = Catalog(child, f"Sub-catalog for {'/'.join(position)}/{child}")
    parent.add_child(child)


def create_item(cat, child, position):
    try:
        create_zarr_item(cat, child, position)
    except Exception as e:
        logger.error(f"Bad dataset: {'/'.join(position)}/{child}: {e}")


def create_zarr_item(cat, child, position):
    logger.debug(f'processing {"/".join(position + [child])}')

    parent = stac_tools.get_subcat(stac_cat, position)
    cg = intake_to_stac.catalog_generator()
    child = cg._gen_item(child, cat[child])
    parent.add_item(child)

In [4]:
def print_subcat(cat, child, position):
    print("/".join(position + [child]))


def print_entry(cat, child, position):
    print("/".join(position + [child]) + ".yaml")


In [5]:
%load_ext autoreload
%aimport intake_to_stac
%autoreload 1


In [ ]:
intake_tools.traverse_tree(
    cat, subcat_callback=create_subcat, entry_callback=create_item
)

DEBUG:PlayWithStack:processing 2D_1h_0.25deg
DEBUG:intake_to_stac:Could not get time range for 2D_1h_0.25deg by loading it. Please supply 'start_datetime' and 'end_datetime' in the metadata.
DEBUG:intake_to_stac:starting to add assets
ERROR:PlayWithStack:Bad dataset: /2D_1h_0.25deg: URI for 2D_1h_0.25deg is a list. Can only handle single files.
DEBUG:PlayWithStack:processing 2D_1h_1deg
DEBUG:intake_to_stac:Could not get time range for 2D_1h_1deg by loading it. Please supply 'start_datetime' and 'end_datetime' in the metadata.
DEBUG:intake_to_stac:starting to add assets
ERROR:PlayWithStack:Bad dataset: /2D_1h_1deg: URI for 2D_1h_1deg is a list. Can only handle single files.
DEBUG:PlayWithStack:processing 2D_1h_native
DEBUG:intake_to_stac:Could not get time range for 2D_1h_native by loading it. Please supply 'start_datetime' and 'end_datetime' in the metadata.
DEBUG:intake_to_stac:starting to add assets
ERROR:PlayWithStack:Bad dataset: /2D_1h_native: URI for 2D_1h_native is a list. Can o

In [7]:
stac_cat.normalize_and_save(
    root_href="/tmp/stac", catalog_type=CatalogType.SELF_CONTAINED
)

In [8]:
sc = Catalog.from_file("/tmp/stac/catalog.json")

In [9]:
print ([ x for x in sc.get_child("FESOM").get_child("IFS_4.4-FESOM_5-cycle3").get_children()])

AttributeError: 'NoneType' object has no attribute 'get_child'

In [ ]:
import pystac
pystac.__version__

'1.12.1'